In [8]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# 配置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 定义网络参数
num_classes = 10
num_epochs = 50
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset  = torchvision.datasets.CIFAR10(root='cs231n/datasets',
                                            train=True,
                                            transform=transforms.ToTensor(),
                                            download=True)
test_dataset  = torchvision.datasets.CIFAR10(root='cs231n/datasets',
                                            train=False,
                                            transform=transforms.ToTensor())

# 定义数据集加载函数 Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(5* 5 * 16, num_classes)

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        #  reshape images to (batch, input_size)
        images = images.to(device)
        labels = labels.to(device)

        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch:[{}/{}], Step:[{}/{}],loss:{:.4f}'.format(epoch+1,num_epochs,i+1,total_steps,loss.item()))

# Test model
model.eval()
# eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(test_loader):
        #  reshape images to (batch, input_size)
        images = images.to(device)
        labels = labels.to(device)

        # forward pass
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('Step:{}, Accuracy of the model on the 100 test images: {}%'.format(i + 1, 100 * correct / total))

# Save the model checkpoint
# torch.save(model.state_dict(), 'model.ckpt')

Files already downloaded and verified
Epoch:[1/50], Step:[100/500],loss:1.6213
Epoch:[1/50], Step:[200/500],loss:1.6138
Epoch:[1/50], Step:[300/500],loss:1.3353
Epoch:[1/50], Step:[400/500],loss:1.4316
Epoch:[1/50], Step:[500/500],loss:1.4473
Epoch:[2/50], Step:[100/500],loss:1.3299
Epoch:[2/50], Step:[200/500],loss:1.4429
Epoch:[2/50], Step:[300/500],loss:1.1939
Epoch:[2/50], Step:[400/500],loss:1.2057
Epoch:[2/50], Step:[500/500],loss:1.3244
Epoch:[3/50], Step:[100/500],loss:1.2071
Epoch:[3/50], Step:[200/500],loss:1.2419
Epoch:[3/50], Step:[300/500],loss:1.1740
Epoch:[3/50], Step:[400/500],loss:1.2663
Epoch:[3/50], Step:[500/500],loss:1.4211
Epoch:[4/50], Step:[100/500],loss:1.0065
Epoch:[4/50], Step:[200/500],loss:1.3180
Epoch:[4/50], Step:[300/500],loss:1.1137
Epoch:[4/50], Step:[400/500],loss:1.1576
Epoch:[4/50], Step:[500/500],loss:1.0275
Epoch:[5/50], Step:[100/500],loss:1.2078
Epoch:[5/50], Step:[200/500],loss:1.2080
Epoch:[5/50], Step:[300/500],loss:1.2287
Epoch:[5/50], Step:

Epoch:[40/50], Step:[200/500],loss:1.0353
Epoch:[40/50], Step:[300/500],loss:0.8195
Epoch:[40/50], Step:[400/500],loss:0.9317
Epoch:[40/50], Step:[500/500],loss:0.9194
Epoch:[41/50], Step:[100/500],loss:0.9530
Epoch:[41/50], Step:[200/500],loss:1.0008
Epoch:[41/50], Step:[300/500],loss:0.8187
Epoch:[41/50], Step:[400/500],loss:0.6679
Epoch:[41/50], Step:[500/500],loss:0.8574
Epoch:[42/50], Step:[100/500],loss:0.7196
Epoch:[42/50], Step:[200/500],loss:0.7119
Epoch:[42/50], Step:[300/500],loss:0.8152
Epoch:[42/50], Step:[400/500],loss:0.9740
Epoch:[42/50], Step:[500/500],loss:0.9848
Epoch:[43/50], Step:[100/500],loss:0.8191
Epoch:[43/50], Step:[200/500],loss:0.9777
Epoch:[43/50], Step:[300/500],loss:1.0141
Epoch:[43/50], Step:[400/500],loss:0.7408
Epoch:[43/50], Step:[500/500],loss:0.8626
Epoch:[44/50], Step:[100/500],loss:0.6844
Epoch:[44/50], Step:[200/500],loss:0.9978
Epoch:[44/50], Step:[300/500],loss:0.7127
Epoch:[44/50], Step:[400/500],loss:1.0172
Epoch:[44/50], Step:[500/500],loss

Step:86, Accuracy of the model on the 100 test images: 63.74418604651163%
Step:87, Accuracy of the model on the 100 test images: 63.724137931034484%
Step:88, Accuracy of the model on the 100 test images: 63.73863636363637%
Step:89, Accuracy of the model on the 100 test images: 63.68539325842696%
Step:90, Accuracy of the model on the 100 test images: 63.75555555555555%
Step:91, Accuracy of the model on the 100 test images: 63.79120879120879%
Step:92, Accuracy of the model on the 100 test images: 63.76086956521739%
Step:93, Accuracy of the model on the 100 test images: 63.784946236559136%
Step:94, Accuracy of the model on the 100 test images: 63.787234042553195%
Step:95, Accuracy of the model on the 100 test images: 63.73684210526316%
Step:96, Accuracy of the model on the 100 test images: 63.75%
Step:97, Accuracy of the model on the 100 test images: 63.72164948453608%
Step:98, Accuracy of the model on the 100 test images: 63.734693877551024%
Step:99, Accuracy of the model on the 100 test